# Particle Lenia [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maxencefaldor/cax/blob/main/examples/22_particle_lenia.ipynb)

## Installation

You will need Python 3.10 or later, and a working JAX installation. For example, you can install JAX with:

In [ ]:
%pip install -U "jax[cuda12]"

Then, install CAX from PyPi:

In [ ]:
%pip install -U "cax[examples]"

## Import

In [ ]:
import jax
import jax.numpy as jnp
import mediapy
from cax.models.particle_lenia import GrowthParams, KernelParams, ParticleLenia, RuleParams, bell_
from flax import nnx

## Configuration

In [2]:
seed = 0

num_dims = 2
num_particles = 200

T = 10

key = jax.random.key(seed)
rngs = nnx.Rngs(seed)

## Init state

In [3]:
def init_state(key):
	state = 12.0 * (jax.random.uniform(key, (num_particles, num_dims)) - 0.5)
	return state

## Model

In [6]:
mean = 4.0
std = 1.0


def compute_weight(mean, std, num_dims):
	"""Compute weight for the kernel."""
	r = jnp.linspace(max(mean - 4 * std, 0.0), mean + 4 * std, 51)
	y = bell_(r, mean, std) * r ** (num_dims - 1)
	s = jnp.trapezoid(y, r) * {2: 2, 3: 4}[num_dims] * jnp.pi
	return 1 / s


weight = compute_weight(mean, std, num_dims)

In [7]:
kernel_params = KernelParams(
	weight=weight,
	mean=mean,
	std=std,
)

growth_params = GrowthParams(
	mean=0.6,
	std=0.15,
)

rule_params = RuleParams(
	c_rep=1.0,
	kernel_params=kernel_params,
	growth_params=growth_params,
)

In [8]:
ca = ParticleLenia(
	num_dims=2,
	T=T,
	rule_params=rule_params,
)

## Visualize

In [11]:
key, subkey = jax.random.split(key)
state = init_state(subkey)

states = ca(state, num_steps=8_192, all_steps=True)

In [12]:
states = jnp.concatenate([state[None], states])
frames = jax.vmap(lambda x: ca.render(x, resolution=512, particle_radius=0.3))(states)

mediapy.show_video(frames, width=256, height=256, fps=600)

This browser does not support the video tag.
